In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

# Model Prompting

In [2]:
from intelligence_layer.core import (
    CompleteInput,
    InMemoryTracer, Llama3InstructModel,
)
from dotenv import load_dotenv

load_dotenv()

tracer = InMemoryTracer()

model = Llama3InstructModel("pharia-1-llm-7b-control")

prompt = model.to_instruct_prompt(
    instruction="Du bist ein Assistent, der Fragen beantwortet.",
    input="Wann wurde Köln gegründet?",
)
complete_input = CompleteInput(prompt=prompt)
response = model.complete(complete_input, tracer=tracer)
response

/Users/jens.luecke/Library/Caches/pypoetry/virtualenvs/rag-qa-workshop-xsC4sNM2-py3.12/lib/python3.12/site-packages/intelligence_layer/core/model.py:229: UserWarning: The provided model is not a recommended model for this model class.Make sure that the model you have selected is suited to be use for the prompt template used in this model class.
  warnings.warn(


No CLIENT_URL specified in environment, using default: https://api.aleph-alpha.com.


CompleteOutput(model_version='.unknown.', completions=[CompletionResult(log_probs=None, completion=' Köln wurde im Jahr 38 v. Chr. von den Römern gegründet und trägt seitdem den offiziellen Namen Colonia Claudia Ara Agrippinensium, abgekürzt CCAA.', completion_tokens=None, finish_reason='end_of_text', raw_completion=None)], num_tokens_prompt_total=35, num_tokens_generated=41, optimized_prompt=None)

In [12]:
tracer.submit_to_trace_viewer()

Open the ]8;id=557316;http://localhost:3001\Trace Viewer]8;;\ to view the trace.

True

# AtMan for Source Highlighting

In [13]:
instruction = (
    "Sie sind ein Assistent, der Filmkritiken klassifiziert. Sie müssen entscheiden, ob die Person den Film mochte oder nicht."
    + ' Die Ausgabe sollte nur aus den Labels "mögen" oder "nicht mögen" bestehen.'
)
user_input = (
    "Nachricht: Ich habe gestern einen Film geschaut. Es ging um einen Agenten, der eine Mission erfüllen musste. Es war wirklich spannend."
    + " Zunächst musste der Agent eine Basis infiltrieren. Dort gab es einen Bösewicht. Blablabla. Lorem Ipsum dolor."
)
prompt = model.to_instruct_prompt(instruction=instruction, input=user_input)
tracer = InMemoryTracer()

complete_input = CompleteInput(prompt=prompt)
complete_output = model.complete(complete_input, tracer)
complete_output.completions[0].completion

' like'

In [14]:
tracer

Open the ]8;id=431457;http://localhost:3001\Trace Viewer]8;;\ to view the trace.

In [15]:
print(prompt.items[0].text)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Sie sind ein Assistent, der Filmkritiken klassifiziert. Sie müssen entscheiden, ob die Person den Film mochte oder nicht. Die Ausgabe sollte nur aus den Labels "mögen" oder "nicht mögen" bestehen.

Nachricht: Ich habe gestern einen Film geschaut. Es ging um einen Agenten, der eine Mission erfüllen musste. Es war wirklich spannend. Zunächst musste der Agent eine Basis infiltrieren. Dort gab es einen Bösewicht. Blablabla. Lorem Ipsum dolor.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [17]:
target = complete_output.completions[0].completion

 like


In [8]:
from intelligence_layer.core import (
    TextHighlight,
    TextHighlightInput,
)
from aleph_alpha_client import PromptGranularity

In [18]:
input = TextHighlightInput(
    rich_prompt=prompt,
    target=target,
    focus_ranges=["input"],
)

highlight_task = TextHighlight(
    model=Llama3InstructModel("pharia-1-llm-7b-control"),
    threshold=0,
    granularity=PromptGranularity.Sentence,
)

tracer = InMemoryTracer()

output = highlight_task.run(input, tracer)
output

/Users/jens.luecke/Library/Caches/pypoetry/virtualenvs/rag-qa-workshop-xsC4sNM2-py3.12/lib/python3.12/site-packages/intelligence_layer/core/model.py:229: UserWarning: The provided model is not a recommended model for this model class.Make sure that the model you have selected is suited to be use for the prompt template used in this model class.
  warnings.warn(


TextHighlightOutput(highlights=[ScoredTextHighlight(start=257, end=305, score=0.05539358559239773), ScoredTextHighlight(start=306, end=365, score=0.01311953342977841), ScoredTextHighlight(start=366, end=391, score=0.059475218214995455), ScoredTextHighlight(start=392, end=442, score=0), ScoredTextHighlight(start=443, end=471, score=0.01020408155649432), ScoredTextHighlight(start=472, end=482, score=0.011953352680464773), ScoredTextHighlight(start=483, end=501, score=0), ScoredTextHighlight(start=501, end=556, score=1.0)])

In [19]:
tracer

Open the ]8;id=678040;http://localhost:3001\Trace Viewer]8;;\ to view the trace.

In [11]:
prompt_text = prompt.items[0].text
for highlight in sorted(
    output.highlights, key=lambda highlight: highlight.score, reverse=True
):
    print("-" * 100)
    print(highlight.score)
    print(prompt_text[highlight.start : highlight.end])

----------------------------------------------------------------------------------------------------
1.0
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
----------------------------------------------------------------------------------------------------
0.4
Es war wirklich spannend.
----------------------------------------------------------------------------------------------------
0.1
Es ging um einen Agenten, der eine Mission erfüllen musste.
----------------------------------------------------------------------------------------------------
0
Nachricht: Ich habe gestern einen Film geschaut.
----------------------------------------------------------------------------------------------------
0.0
Zunächst musste der Agent eine Basis infiltrieren.
----------------------------------------------------------------------------------------------------
0.0
Dort gab es einen Bösewicht.
----------------------------------------------------------------------------------------------------
